In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
import pickle
import tensorflow as tf

import fasttext
import fasttext.util

# Hyperlex load (i dont use yet)


In [5]:
file = "././data/hyperlex-data/hyperlex-all.txt"

results = []
with open(file) as f:
    line = f.readline()
    while line:
        results.append(line.strip().split(" "))
        line = f.readline()
f.close()

HyperLex = pd.DataFrame(results)
HyperLex.columns = HyperLex.iloc[0]
HyperLex = HyperLex.iloc[1:].reset_index(drop=True)

HyperLex.head()


,WORD1,WORD2,POS,TYPE,AVG_SCORE,AVG_SCORE_0_10,STD,SCORES..,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,conflict,disagreement,N,r-hyp-1,5.20,8.67,1.25,5,6,6,...,6,5,2,6,4,6,None,None,None,None
1,advance,take,V,no-rel,1.42,2.37,1.66,3,0,3,...,0,5,0,0,1,2,3,0,None,None
2,trail,follow,V,hyp-2,4.31,7.18,1.86,4,4,6,...,6,6,6,3,0,4,3,6,6,None
3,mason,worker,N,hyp-3,4.50,7.5,1.76,6,6,4,...,6,3,6,3,5,0,5,6,None,None
4,aura,light,N,hyp-1,3.69,6.15,1.86,4,5,3,...,6,0,3,6,3,0,4,5,5,None


# load baroni dataset

In [7]:
neg_file = "././data/eacl2012-data/negative-examples.txtinput"
pos_file = "././data/eacl2012-data/positive-examples.txtinput"
filenames = ["neg_file", "pos_file"]


for i, file in enumerate([neg_file, pos_file]):
    globals()['results_{}'.format(filenames[i])] = []
    
    with open(file) as f:
        line = f.readline()
        while line:
            globals()['results_{}'.format(filenames[i])].append(line.replace("-n", "").replace("\n", "").strip("").split("\t"))
            line = f.readline()
    f.close()


In [66]:
baroni = sum(results_neg_file, []) + sum(results_pos_file, [])
set_baroni = set(baroni)

print("the length of all words in baroni: ", len(all_words))
print("number of unique in baroni: ", len(set(all_words)))


the length of all words in baroni:  5540
number of unique in baroni:  1478


# load wikipedia data(sub)set 
Here I take a subset from the wikipedia data and create a method to substract a set of context vectors.

I found the most common word in the 4 text is "church", i use that for further testing.

In [68]:
# ran this on the workstation and copied it into the notebook
wikidata = {'title': ['Yangliuqing', 'Orana Australia Ltd', "St. Mary's Church, Sønderborg"], 'text': ['Yangliuqing () is a market town in Xiqing District, in the western suburbs of Tianjin, People\'s Republic of China. Despite its relatively small size, it has been named since 2006 in the "famous historical and cultural market towns in China".\n\nIt is best known in China for creating nianhua or Yangliuqing nianhua. For more than 400 years, Yangliuqing has in effect specialised in the creation of these woodcuts for the New Year.  wood block prints using vivid colourschemes to portray traditional scenes of children\'s games often interwoven with auspiciouse objects.\n\n, it had 27 residential communities () and 25 villages under its administration.\n\nShi Family Grand Courtyard\n\nShi Family Grand Courtyard (Tiānjīn Shí Jiā Dà Yuàn, 天津石家大院) is situated in Yangliuqing Town of Xiqing District, which is the former residence of wealthy merchant Shi Yuanshi - the 4th son of Shi Wancheng, one of the eight great masters in Tianjin. First built in 1875, it covers over 6,000 square meters, including large and small yards and over 200 folk houses, a theater and over 275 rooms that served as apartments and places of business and worship for this powerful family. Shifu Garden, which finished its expansion in October 2003, covers 1,200 square meters, incorporates the elegance of imperial garden and delicacy of south garden. Now the courtyard of Shi family covers about 10,000 square meters, which is called the first mansion in North China. Now it serves as the folk custom museum in Yangliuqing, which has a large collection of folk custom museum in Yanliuqing, which has a large collection of folk art pieces like Yanliuqing New Year pictures, brick sculpture.\n\nShi\'s ancestor came from Dong\'e County in Shandong Province, engaged in water transport of grain. As the wealth gradually accumulated, the Shi Family moved to Yangliuqing and bought large tracts of land and set up their residence. Shi Yuanshi came from the fourth generation of the family, who was a successful businessman and a good household manager, and the residence was thus enlarged for several times until it acquired the present scale. It is believed to be the first mansion in the west of Tianjin.\n\nThe residence is symmetric based on the axis formed by a passageway in the middle, on which there are four archways. On the east side of the courtyard, there are traditional single-story houses with rows of rooms around the four sides, which was once the living area for the Shi Family. The rooms on north side were the accountants\' office. On the west are the major constructions including the family hall for worshipping Buddha, theater and the south reception room. On both sides of the residence are side yard rooms for maids and servants.\n\nToday, the Shi mansion, located in the township of Yangliuqing to the west of central Tianjin, stands as a surprisingly well-preserved monument to China\'s pre-revolution mercantile spirit. It also serves as an on-location shoot for many of China\'s popular historical dramas. Many of the rooms feature period furniture, paintings and calligraphy, and the extensive Shifu Garden.\n\nPart of the complex has been turned into the Yangliuqing Museum, which includes displays focused on symbolic aspects of the courtyards\'  construction, local folk art and customs, and traditional period furnishings and crafts.\n\nSee also \n\nList of township-level divisions of Tianjin\n\nReferences \n\n http://arts.cultural-china.com/en/65Arts4795.html\n\nCategory:Towns in Tianjin', 'Orana Australia Ltd is a not-for-profit organisation that provides a diverse range of training and support services to over 650 people with disabilities and their families in South Australia.\n\nHistory\nThe Mentally Retarded Children’s Society of SA Inc. was established in 1950 by a group of parents who wanted education, employment and accommodation opportunities for their children within the local community at a time when institutionalised care in Adelaide was their only alternative.\n\nThe society’s aims were to seek education or training facilities for people with intellectual disabilities, to establish sheltered workshops, and to establish residential hostels.\n\nA number of sheltered workshops were established, and in 1980, the name was changed to the Aboriginal word "Orana", which means "Welcome".\n\nToday, Orana provides assisted employment, assisted accommodation and respite services to people with intellectual disabilities.\n\nOrana\'s current and previous clients include Mitsubishi Motors, Clipsal, RAA, Elders Limited, and Billycart Kids.\n\nOrana was one of the first disability service organisations to achieve Quality Accreditation.  The services and products they offer are:\n\n Packaging\n Assembly\n Sewing\n Collating & Mailing\n Furniture - Retail\n Furniture – Manufacture for Commercial Market\n Worm Farming\n Work Crews\n Pet & Grain – Retail\n\nIn 2018, after 65 years of bettering people’s lives, Orana identified a community need and expanded their operations into the aged care sector.\n\nAfter the unveiling of the Australian Government’s Commonwealth Home Support Programme (CHSP) and seeing it as a natural step of progression, Orana now provides quality tailored aged care at home.\n\nThe well-resourced organization delivers help across a range of areas, helping the elderly remain where they want to be - in the comfort of their own home during their later years.\n\nOrana continues with its mission to support people remain independent, valued and productive members of the community.\n\nReferences\n\nExternal links \n \n\nCategory:Disability organisations based in Australia\nCategory:Organisations based in South Australia', "The St. Mary's Church is a church owned by the Church of Denmark in Sønderborg, Denmark and the church of the parish with the same name. Thanks to its location on a hill, the church building is very iconic for the city.\n\nHistory \nIn the Middle Ages there was a leper colony on a hill just outside the city. It was named after Saint George and around 1300 the chapel of this leper colony stood in the place of the present St. Mary's Church. After the old parish church of the city, the St. Nicholas Church, was demolished around 1530, the Saint-George chapel became the new main church. Towards the end of the 16th century, John II, Duke of Schleswig-Holstein-Sonderburg commissioned the enlargement of the building in order to make it suitable for the function of the parish church of his city.\n\nThe current St. Mary's Church \nIn 1595 a start was made on the partial demolition of the old church and the construction of the new church. Only parts of the old medieval church remained. From the medieval church, a medieval wooden wall cupboard dating from about 1400 remained. The solemn inauguration of the new parish church took place just before Christmas in 1600. In 1649 the George Church was renamed as the Mary Church. The name of Saint George stayed in the Danish names Sankt Jørgensgade and Jørgensbjerg.\n\nReferences \n\nCategory:Buildings and structures in Sønderborg Municipality\nCategory:Churches in Denmark\nCategory:Church of Denmark churches"]}

### Pre processing the wikipedia dataset

In [10]:
import re
import numpy as np  

def text_preprocessing(
    text:list,
    punctuations = r'''!()-[]{};:'"\,<>./?@#$%^&*_“~''',
    stop_words=['and', 'a', 'is', 'the', 'in', 'be', 'will']
    )->list:
    """
    A method to preproces text
    """
    for x in text.lower(): 
        if x in punctuations: 
            text = text.replace(x, "")

    # Removing words that have numbers in them
    text = re.sub(r'\w*\d\w*', '', text)

    # Removing digits
    text = re.sub(r'[0-9]+', '', text)

    # Cleaning the whitespaces
    text = re.sub(r'\s+', ' ', text).strip()

    # Setting every word to lower
    text = text.lower()

    # Converting all our text to a list 
    text = text.split(' ')

    # Droping empty strings
    text = [x for x in text if x!='']

    # Droping stop words
#     text = [x for x in text if x not in stop_words]

    return text

### apply preprocessing to wikidata

In [72]:
# Cleaning the text
texts = [x for x in wikidata['text']]

wiki_all_text = []

for text in texts:
    # Appending to the all text list
    text = text_preprocessing(text)
    wiki_all_text += text

### Creating a context dictionairy from wiki subset 

In [70]:
# Defining the window for context
window = 2

# Creating a dictionary entry for each word in the texts
context_dict = { i : list() for i in set(all_text)}


for i, word in enumerate(all_text):
    for w in range(window):
        # Getting the context that is ahead by *window* words
        
        if i + 1 + w < len(all_text):
            context_dict[word].append(all_text[(i + 1 + w)]) 
        # Getting the context that is behind by *window* words    
        if i - w - 1 >= 0:
            context_dict[word].append(all_text[(i - w - 1)])


# use fasttext to get word vectors


In [38]:
# get pre trained fastext model code is now replaced with a load file

# ft = fasttext.load_model('./data/cc.en.300.bin')
# ft.get_dimension()
# fasttext.util.reduce_model(ft, 100)
# ft.get_dimension()

# ft.save_model("ft_reduced_100.bin")

In [2]:
ft = fasttext.load_model("././Data/ft_reduced_100.bin")

# EMPIRICAL COVARIANCES
the actual implementation of the equation

In [74]:
# Extract words that appear in both baroni as in wikidata subset
combined_set = set(wiki_all_text)&set(set_baroni)
print(combined_set)

{'district', 'water', 'furniture', 'disability', 'hall', 'size', 'set', 'community', 'art', 'pet', 'wall', 'room', 'merchant', 'end', 'wood', 'church', 'ancestor', 'area', 'worm', 'name', 'organization', 'chapel', 'building', 'people', 'son', 'passageway', 'list', 'collection', 'care', 'theater', 'business', 'range', 'businessman', 'word', 'work'}


In [76]:
total = np.zeros((100,100))

test_words = ['church', 
              'is',
              'marys',
              'owned',
              'a',
              'of',
              'the',
              'of',
              'the',
              'building',
              'the',
              'after',
              'chapel',
              'of', 
              'the', 
              'became', 
              'saintgeorge']

covariance = {}

for word in test_words:
    for c_word in context_dict[word]:
        w = ft.get_word_vector(word)
        total += np.outer((ft.get_word_vector(c_word) - w), (ft.get_word_vector(c_word) - w))
    
    covariance[word] = (total / (len(context_dict[word]) * window))


In [77]:
covariance

{'church': array([[ 0.04562442, -0.02122848, -0.01190691, ..., -0.01121585,
          0.01134437,  0.01193672],
        [-0.02122848,  0.03855264,  0.01497514, ...,  0.01447863,
         -0.01185497, -0.02005979],
        [-0.01190691,  0.01497514,  0.01715384, ...,  0.00313075,
         -0.00498252, -0.00924604],
        ...,
        [-0.01121585,  0.01447863,  0.00313075, ...,  0.00932712,
         -0.00440523, -0.00687782],
        [ 0.01134437, -0.01185497, -0.00498252, ..., -0.00440523,
          0.00654882,  0.00697117],
        [ 0.01193672, -0.02005979, -0.00924604, ..., -0.00687782,
          0.00697117,  0.01200381]]),
 'is': array([[ 0.12123077, -0.10011089, -0.03396592, ..., -0.05859231,
          0.03339109,  0.04640988],
        [-0.10011089,  0.20091142,  0.04774174, ...,  0.10044581,
         -0.04983195, -0.09066189],
        [-0.03396592,  0.04774174,  0.0396514 , ...,  0.01942524,
         -0.01307911, -0.0253555 ],
        ...,
        [-0.05859231,  0.10044581,  0.

In [78]:
from numpy import random

mean = ft.get_word_vector('church')
covariance_matrix = covariance["church"]

mean2 = ft.get_word_vector('building')
covariance_matrix2 = covariance["building"]

p = random.multivariate_normal(mean, covariance_matrix)
q = random.multivariate_normal(mean2, covariance_matrix2)

torch.nn.functional.kl_div(torch.from_numpy(p), torch.from_numpy(q))

tensor(0.4182, dtype=torch.float64)

In [ ]:

# mean = torch.from_numpy(mean)
# covariance_matrix = torch.from_numpy(covariance_matrix)


# p = torch.distributions.multivariate_normal.MultivariateNormal(mean, covariance_matrix=covariance_matrix)

# loc = np.reshape(ft.get_word_vector("chapel"), (10, 10))
# covariance_matrix = np.reshape(covariance["chapel"], (10, 10))
# loc = torch.from_numpy(mean)
# covariance_matrix = torch.from_numpy(covariance_matrix)

# q = torch.distributions.MultivariateNormal(loc, covariance_matrix)



# torch.distributions.kl.kl_divergence(torch.from_numpy(p), torch.from_numpy(q))
